### Data preprocessing

In [1]:
import mne
import numpy as np
print(mne.__version__)
print(np.__version__)
import sys
sys.path.append('../src/utils/data_preprocessing')
sys.path.append('../src/model')

import temp_utils as utils
from  svm import train_svm_model

1.6.0
1.26.3


In [2]:
file_path = r'../src/data/test_data/16/16.edf'
raw = utils.read_edf(file_path)
raw = utils.rename_edf(raw)
with mne.utils.use_log_level('WARNING'):
        raw.filter(1, 60, fir_design='firwin')

picks = utils.obtain_picks(raw)
info = utils.obtain_eeg_info(raw, picks)

Extracting EDF parameters from d:\Research\EEG\EEG_DL_Classifier\src\data\test_data\16\16.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3604471  =      0.000 ...  3604.471 secs...


d:\Research\EEG\EEG_DL_Classifier\notebooks\../src/utils/data_preprocessing\temp_utils.py:15: RuntimeWarning: Scaling factor is not defined in following channels:
POL SpO2, POL EtCO2, POL Pulse, POL CO2Wave
  raw = mne.io.read_raw_edf(file_path, preload=True, encoding='latin1')
d:\Research\EEG\EEG_DL_Classifier\notebooks\../src/utils/data_preprocessing\temp_utils.py:15: RuntimeWarning: Physical range is not defined in following channels:
POL SpO2, POL EtCO2, POL Pulse, POL CO2Wave
  raw = mne.io.read_raw_edf(file_path, preload=True, encoding='latin1')


In [3]:
utils.obtain_multi_topomap(raw, picks, info, '16', fig_size=128, is_energy=True, is_test=False)

Jie Log: output_dir: d:\Research\EEG\EEG_DL_CLASSIFIER\src\data\output_image\16


Obtain topomap for 16: 100%|██████████| 3600/3600 [04:00<00:00, 14.98it/s]


In [ ]:
from PIL import Image
import os

# 文件夹路径
main_path = r'D:\Research\EEG\EEG_DL_Classifier\tests'
eeg_file_name = '1'
folder_path = os.path.join(main_path, eeg_file_name)

output_path = r'D:\Research\EEG\EEG_DL_Classifier\tests\eeg.gif'


# 读取所有图像文件
images = []
for i in range(3600):
    file_name = f'{eeg_file_name}_{i}.png'  # 假设文件命名方式为 "0.png", "1.png", ..., "3599.png"
    file_path = os.path.join(folder_path, file_name)
    if os.path.exists(file_path):
        img = Image.open(file_path)
        images.append(img)




In [ ]:
gif_path = output_path
images[0].save(gif_path, save_all=True, append_images=images[0:], optimize=False, duration=100, loop=1)

In [ ]:
egg_filename = '1'
X_train, X_test, y_train, y_test = utils.prepare_data(egg_filename, is_test = False)

### Data Distribution

In [ ]:
import pandas as pd


# figure out whether the number of 0 is more than 1 in the label_df
def which_is_more(label_df):
    label_df = label_df['label'].values
    if (sum(label_df == 0) / (sum(label_df == 1)+sum(label_df == 0)) ) > 0.8:
            return 0
    elif (sum(label_df == 1) / (sum(label_df == 1)+sum(label_df == 0)) ) > 0.8:
            return 1

index = 33
normal_index, patient_index = [], []
for i in range(1,index + 1):
    eeg_file_name = str(i)
    with pd.ExcelFile(rf'../src/data/EEG_DATASET/{eeg_file_name}/{eeg_file_name}.xlsx') as xls:
        label_df = pd.read_excel(xls, 'Sheet1',header=None)
        label_df.rename(columns={0: 'label'}, inplace=True)
    
    if which_is_more(label_df) == 0:
        normal_index.append(i)
    else:
        patient_index.append(i)


print(f'病人: {patient_index}')
print(f'正常人: {normal_index}')

       


### Model Trainning

In [4]:
eeg_file_names = ['1','16']
X_train, X_test, y_train, y_test = utils.prepare_data_from_multi_file(eeg_file_names, handle_data_imbalance=False, is_test = False)

Jie log: all_images_np.shape: (7200, 16384), all_labels_np.shape: (7200,)


In [5]:
train_svm_model(X_train, y_train, X_test, y_test)

Training SVM model...
X_train shape: (5040, 16384)
y_train shape: (5040,)
X_test shape: (2160, 16384)
y_test shape: (2160,)
Accuracy: 0.8462962962962963
Recall: 0.847265221878225
Precision: 0.8169154228855722
F1 Score: 0.8318135764944277
Confusion Matrix:
[[1007  184]
 [ 148  821]]


SVC()